In [ ]:
import sys
if "google.colab" in sys.modules:
    print("Colab Detected")
    import tensorflow as tf
    gpus = tf.config.list_physical_devices("GPU")
    if not gpus:
        raise RuntimeError("Nessuna GPU trovata.Controlla di aver selezionato il runtime giusto.")
    else:
        print(f"Trovate {len(gpus)} GPU:\n{gpus}")
    
    !git clone https://github.com/AtomicDonuts/Progetto_Computings.git
    %cd Progetto_Computings/
    !pip install -q -r requirements.txt
    !python3 fits_import/fits2csv.py
    
    sys.path.append("imports/")
    import custom_variables as custom_paths
else:
    print("Local Machine Detected")
    sys.path.append("../imports/")
    import custom_variables as custom_paths

In [ ]:
from keras.layers import Dense, Input, Concatenate, Flatten, Dropout
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
df = pd.read_csv(custom_paths.csv_path)
df = df[(df["CLASS_GENERIC"] == "AGN") | (df["CLASS_GENERIC"] == "Pulsar")]
print(len(df))

flux_band = np.array([[f"Flux_Band_{i}", f"Sqrt_TS_Band_{i}"] for i in range(8)])

flux_hist = np.array([[f"Flux_History_{i}", f"Sqrt_TS_History_{i}"] for i in range(14)])

norm_cols = np.array(
    [[f"Flux_Band_{i}", f"Sqrt_TS_Band_{i}"] for i in range(8)]
    + [[f"Flux_History_{i}", f"Sqrt_TS_History_{i}"] for i in range(14)]
).flatten()

scaler = StandardScaler()
scaler.fit(df[norm_cols])
scaled_data = scaler.transform(df[norm_cols])
df[norm_cols] = scaled_data

flux_band = np.dstack((df[flux_band[:, 0]].to_numpy(), df[flux_band[:, 1]].to_numpy()))
flux_hist = np.dstack((df[flux_hist[:, 0]].to_numpy(), df[flux_hist[:, 1]].to_numpy()))
print(f"Flux_Band Size: {flux_band.shape}")
print(f"Flux_History Size: {flux_hist.shape}")
is_agn = df["CLASS_GENERIC"].to_numpy() == "AGN"
labelkfold = np.zeros(len(df),dtype=int)
labelkfold[is_agn] = 1
labels = np.zeros((len(df), 2), dtype=int)
labels[is_agn, 0] = 1
labels[~is_agn, 1] = 1

In [ ]:
acc_per_fold = []
loss_per_fold = []
fold_no = 1
skf = StratifiedKFold(n_splits=10,shuffle=True)
for train,test in skf.split(np.zeros(len(labels)),labelkfold):

    inputs = Input(shape=flux_band.shape[1:])
    hidden = Flatten()(inputs)
    hidden = Dense(16, activation="relu")(hidden)
    inputs2 = Input(shape=flux_hist.shape[1:])
    hidden2 = Flatten()(inputs2)
    hidden2 = Dense(16, activation="relu")(hidden2)
    concat = Concatenate()([hidden2, hidden])
    final_hidden = Dense(4, activation="relu")(concat)
    outputs = Dense(2, activation="softmax")(final_hidden)
    model_paper = Model(inputs=[inputs, inputs2], outputs=outputs)
    model_paper.compile(loss="categorical_crossentropy",
                        optimizer="adam",
                        metrics = ["accuracy"])
    model_paper.optimizer.learning_rate = 0.01
    print("------------------------------------------------------------------------")
    print(f"Training for fold {fold_no} ...")
    # early_stopping = EarlyStopping(
    #     monitor="accuracy", patience=10, restore_best_weights=True
    # )
    # reduce_lr = ReduceLROnPlateau(monitor="accuracy", factor=0.5, patience=5)

    history = model_paper.fit(
        [flux_band[train], flux_hist[train]],
        labels[train],
        #batch_size=64,
        validation_split=0.7,
        epochs=50,
    )
    scores = model_paper.evaluate(
        [flux_band[test], flux_hist[test]], labels[test], verbose=0
    )
    print(f'Score for fold {fold_no}: {model_paper.metrics_names[0]} of {scores[0]}; {model_paper.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    fold_no = fold_no + 1

In [ ]:
# == Provide average scores ==
print("------------------------------------------------------------------------")
print("Score per fold")
for i in range(0, len(acc_per_fold)):
    print("------------------------------------------------------------------------")
    print(f"> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%")
print("------------------------------------------------------------------------")
print("Average scores for all folds:")
print(f"> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})")
print(f"> Loss: {np.mean(loss_per_fold)}")
print("------------------------------------------------------------------------")